In [1]:
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('./datasets/bayes/spam.csv', encoding='ISO-8859-1')

In [7]:
le = LabelEncoder()

In [8]:
data = df.to_numpy()

In [14]:
X = data[:, 1]
y = data[:, 0]

In [23]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [46]:
# for one sentence
def getStemmedReview(review):
    review = review.lower()
    
    # tokenize
    tokens = tokenizer.tokenize(review)
    
    new_tokens = [token for token in tokens if token not in sw]
    
    stemmed_words = [ps.stem(token) for token in new_tokens]
    
    cleaned_review = ' '.join(stemmed_words)
    
    return cleaned_review

In [47]:
# for entire document/paragraph
def getStemmedDocument(document):
    d = []
    for docu in document:
        d.append(getStemmedReview(docu))
    return d

In [48]:
stemmed_document = getStemmedDocument(X)

In [51]:
stemmed_document[10]

'gonna home soon want talk stuff anymor tonight k cri enough today'

In [52]:
X[10]

"I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today."

In [53]:
cv = CountVectorizer()

In [54]:
vectorised_corpus = cv.fit_transform(stemmed_document)

In [56]:
X = vectorised_corpus.todense()

In [58]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [59]:
from sklearn.naive_bayes import MultinomialNB

In [60]:
m = MultinomialNB()
m.fit(X_train, y_train)
m.score(X_test, y_test)

0.977705274605764

In [61]:
messages = [
        """
Hi Kunal,
We invite you to participate in MishMash - India’s largest online diversity hackathon. 
The hackathon is a Skillenza initiative and sponsored by Microsoft, Unity, Unilever, Gojek, Rocketium and Jharkhand Government. 
We have a special theme for you - Deep Tech/Machine Learning - sponsored by Unilever, which will be perfect for you.""",
    
    
    """Join us today at 12:00 PM ET / 16:00 UTC for a Red Hat DevNation tech talk on AWS Lambda and serverless Java with Bill Burke.
Have you ever tried Java on AWS Lambda but found that the cold-start latency and memory usage were far too high? 
In this session, we will show how we optimized Java for serverless applications by leveraging GraalVM with Quarkus to 
provide both supersonic startup speed and a subatomic memory footprint.""",

    """We really appreciate your interest and wanted to let you know that we have received your application.
There is strong competition for jobs at Intel, and we receive many applications. As a result, it may take some time to get back to you.
Whether or not this position ends up being a fit, we will keep your information per data retention policies, 
so we can contact you for other positions that align to your experience and skill set.
"""
]

In [62]:
def prepare_message(messages):
    d = getStemmedDocument(messages)
    return cv.transform(d) # dont do fit transform for test data because it will create vocab again and old data will be lost.

In [63]:
clean_messages = prepare_message(messages)

In [65]:
y_pred = m.predict(clean_messages)
print(y_pred)

['ham' 'spam' 'ham']
